In [10]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re

# === Авторизация ===
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("limehd-451312-44fe1c22414e.json", scope)
client = gspread.authorize(creds)
sheet = client.open_by_key("1tE8lfckXvX536H2D64L4coqYRbZpl8eMhQQ2N6gAchw").worksheet("За день")

# === Парсинг русских дат вручную ===
def parse_russian_date(date_str):
    # Удаляем "г.", узкие пробелы, переводим в нижний регистр
    clean_str = str(date_str).replace('\u202f', ' ').replace("г.", "").strip().lower()

    # Обновлённый словарь с сокращёнными месяцами
    months = {
        'янв.': '01', 'фев.': '02', 'мар.': '03', 'апр.': '04',
        'мая': '05', 'май.': '05', 'июн.': '06', 'июл.': '07',
        'авг.': '08', 'сен.': '09', 'окт.': '10', 'ноя.': '11', 'дек.': '12',
        'января': '01', 'февраля': '02', 'марта': '03', 'апреля': '04',
        'июня': '06', 'июля': '07', 'августа': '08', 'сентября': '09',
        'октября': '10', 'ноября': '11', 'декабря': '12'
    }

    match = re.match(r"(\d{1,2})\s+(\w+\.?)\s+(\d{4})", clean_str)
    if not match:
        return None
    day, month_rus, year = match.groups()
    month = months.get(month_rus)
    if not month:
        return None
    return f"{int(day):02d}.{month}.{str(year)[2:]}"

# === Чтение заголовков таблицы (даты в строке 1) ===
header = sheet.row_values(1)
date_to_col = {}
for col_idx, val in enumerate(header, start=1):
    try:
        dt = pd.to_datetime(val, dayfirst=True)
        formatted = dt.strftime("%d.%m.%y")
        date_to_col[formatted] = col_idx
    except:
        continue

# === Загрузка и обработка файла ===
df = pd.read_csv("daily_performance.csv")
df["date"] = df["Date"].apply(parse_russian_date)
df["conversion"] = df["Коэффициент конверсии страницы приложения: Все страны или регионы"]\
    .str.replace(",", ".").str.replace(" %", "").astype(float)

# === Обновление строки 10 ===
for _, row in df.iterrows():
    date = row["date"]
    value = f"{row['conversion']:.2f}".replace(".", ",")

    if date in date_to_col:
        col = date_to_col[date]
        current_val = sheet.cell(10, col).value
        if current_val is None:
            current_val = ""
        else:
            current_val = current_val.strip().replace(",", ".")

        if current_val == "":
            needs_update = True
        else:
            try:
                needs_update = abs(float(current_val) - float(value.replace(",", "."))) > 0.01
            except:
                needs_update = True

        if needs_update:
            sheet.update_cell(10, col, value)
            print(f"Обновлено {date} (столбец {col}): {value}")
        else:
            print(f"Уже актуально {date}")
    else:
        print(f"Дата {date} не найдена в заголовке")



Уже актуально 17.05.25
Уже актуально 18.05.25
Уже актуально 19.05.25
Уже актуально 20.05.25
Уже актуально 21.05.25
Уже актуально 22.05.25
Уже актуально 23.05.25
Уже актуально 24.05.25
Уже актуально 25.05.25
Уже актуально 26.05.25
Уже актуально 27.05.25
Уже актуально 28.05.25
Уже актуально 29.05.25
Уже актуально 30.05.25
Уже актуально 31.05.25
Уже актуально 01.06.25
Уже актуально 02.06.25
Уже актуально 03.06.25
Уже актуально 04.06.25
Уже актуально 05.06.25
Уже актуально 06.06.25
Уже актуально 07.06.25
Уже актуально 08.06.25
Уже актуально 09.06.25
Уже актуально 10.06.25
Обновлено 11.06.25 (столбец 99): 7,80
Обновлено 12.06.25 (столбец 100): 10,78
Обновлено 13.06.25 (столбец 101): 11,62
Обновлено 14.06.25 (столбец 102): 12,14
Обновлено 15.06.25 (столбец 103): 11,45
